# 1.3 Retrieval

In this notebook we will use a toy search to retrieve data from the FAQs for the purpose of providing context to the LLM down the line.

In [ ]:
import minsearch # Toy search module created by DataTalksClub
import json

In [10]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [11]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [12]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

## Indexing the documents

In [9]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [10]:
index.fit(documents)

## Searching the documents using Minsearch

In [29]:
q = 'The course has already started, can I still enroll?'

In [ ]:
#boost allows you to dictate the importance of fields when looking for search results.
boost = {
    'question': 3.0,
    'section': 0.5,
}

results = index.search(
    query=q,
    boost_dict=boost,
    filter_dict={'course': 'data-engineering-zoomcamp'}, # limit search to a particular course
    num_results=5
)

In [18]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

# 1.4 Generation with OpenAI

## Config connection to OpenAI

In [25]:
from openai import OpenAI
import os
from dotenv import dotenv_values


In [26]:
secrets_dir = os.path.expanduser("~/Desktop/projects/.secrets")

config = {
    **dotenv_values(secrets_dir + "/.env.llm-zc"),
}

OPENAI_KEY = config.get("OPEN_AI_API_KEY")

In [27]:
client = OpenAI(
    api_key=OPENAI_KEY
)

In [30]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

In [31]:
response.choices[0].message.content #gives a generic response as it does not have context yet.

"It's best to contact the course administrator or the institution offering the course to get the most accurate information about late enrollment. Some courses allow students to enroll after the start date, often with certain conditions or up until a specific deadline. Make sure to inquire about any potential late fees, catch-up requirements, or prerequisites you might need to complete."

Now we will use the responses from the knowledge base to get more specific response from open ai.

In [1]:
prompt_template = """
You're a course teaching assistant. You support prospective and current students, by answering questions they have about the courses we offer.
Answer the QUESTION based on the CONTEXT provided from our FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: {question}

CONTEXT: 
{context}
"""

In [2]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

NameError: name 'results' is not defined

In [ ]:
print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start wit

In [34]:
prompt = prompt_template.format(question=q, context=context).strip()

In [ ]:
prompt

In [35]:
print(prompt)

You're a course teaching assistant. You support prospective and current students, by answering questions they have about the courses we offer.
Answer the QUESTION based on the CONTEXT provided from our FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: The course has already started, can I still enroll?

CONTEXT: 
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also contin

In [ ]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)

response.choices[0].message.content

"Yes, even if you don't register before the course start date, you are still eligible to submit the homeworks. However, be aware that there will be deadlines for turning in the final projects, so it's important not to leave everything for the last minute."

## 1.5 Modularising the code

In [29]:
def search(query):
    #boost allows you to dictate the importance of fields when looking for search results.
    boost = {
        'question': 3.0,
        'section': 0.5,
    }

    results = index.search(
        query=query,
        boost_dict=boost,
        filter_dict={'course': 'data-engineering-zoomcamp'}, # limit search to a particular course
        num_results=5
    )

    return results

In [32]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. You support prospective and current students, by answering questions they have about the courses we offer.
    Answer the QUESTION based on the CONTEXT provided from our FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contain the answer, output NONE.

    QUESTION: {question}

    CONTEXT: 
    {context}
    """.strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
        
    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [33]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [34]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [ ]:
query = "The course has started, can I still enroll?"
rag(query)

'Yes, you can still enroll in the course even if it has already started. However, keep in mind the deadlines for submitting final projects.'

# 1.6 Implementing Elastic Search

First we need to index the documents again for the elastic search.



In [1]:
from elasticsearch import Elasticsearch

In [2]:
es_client = Elasticsearch("http://localhost:9200")

In [3]:
es_client.info()

ObjectApiResponse({'name': '13aa71550bdf', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'NCsO1K4yR7SQkpRJtKTRSQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [4]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(
    index=index_name,
    body=index_settings
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [5]:
from tqdm.auto import tqdm

In [13]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [15]:
# Now we can query the newly index Elastic Search DB
query = 'I just discovered the course. Can I still enroll?'

In [16]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

In [18]:
response = es_client.search(index=index_name, body=search_query)

In [20]:
result_docs = []

for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

In [22]:
result_docs

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at

Turn elastic search into a function so that we can substitute it into the RAG workflow.

In [23]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    #search through the documents using the parameters above
    response = es_client.search(index=index_name, body=search_query)

    result_docs = [] # list to house search results

    for hit in response['hits']['hits']: # Obtain search results in format ready to be ingested in RAG workflow, without other elastic search metadata
        result_docs.append(hit['_source'])

    return result_docs

In [24]:
elastic_search(query)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at

In [35]:
def es_rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [36]:
es_rag(query)

"Yes, even if you don't register, you can still enroll and submit the homework. However, be aware that there will be deadlines for turning in the final projects."